<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/PPK_trajectory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PPK / PPP / RTX GNSS Trajectory Analysis ToolBox
===

See store at: www.lidar.net

For use with the [CWW-PPK](http://lidar.net) Precision GNSS system designed for UAS & Manned Structure from Motion data collection.  Precision GNSS trajectory analysis tool.  Examine and compare PPK & PPP GNSS trajectories.


<<<-- Click the arrow to the left to Open the Tool Cells.  
===

In [ ]:
#@title Mount your Google Drive as /content/Gdrive 
from google.colab import drive
!rm -rf /content/sample_data/
drive.mount('/content/.gdrive')
!ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
print('Your Gdrive is mounted')

In [ ]:
#@title **Initialize Tool Box:** Run this Cell first to initilize the system.  Function defs are located inside this cell.  Double-click here to see the functions.

import pandas as pd
import os
import pathlib
import numpy as np
import ipywidgets as widgets
from google.colab import widgets

import sys
import importlib
import subprocess

import calendar
import datetime
import platform
import re
import urllib
import matplotlib as mp
import matplotlib.pyplot as plt

from collections import OrderedDict
from pathlib import Path, PureWindowsPath
from matplotlib import colors
from pandas.plotting import register_matplotlib_converters

import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from IPython.core.display import HTML, display
from IPython.display import display

import ipywidgets as widgets
from google.colab import widgets as gwidgets
########################################

TOOLS = 'pan,wheel_zoom,hover,box_zoom,reset,undo, redo'

########################################################
#   Begin configuring the Bokeh plotting tools         #
########################################################
def init_analysis():
  global settings,  trajectories, Trj, binSize_In_cm, binsz,c, Trj_fn
  global Trj1, Trj2, Trj3, Trj4, Trj5, Trj6, Trj7, Trj8, Trj9, Trj10

  try:
    settings
    trajectories
  except NameError:
    trajectories = {}
    for i in range(1,11):
      trajectories[i] = {}
    settings = {}
    settings['binsz']       = 0.025
    settings['plot_width']  = 400
    settings['plot_height'] = 400

    binSize_In_cm = 2.5
    binsz = binSize_In_cm / 100.0
    settings['binsz'] = binsz
    Trj = {}
    Trj1 = Trj2 = Trj3 = Trj4 = Trj5 = Trj6 = Trj7 = Trj8 = Trj9 = Trj10 = False
    Trj[1] = Trj1; Trj[2] = Trj2; Trj[3] = Trj3; Trj[4] = Trj4; Trj[5] = Trj5; Trj[6] = Trj6; Trj[7] = Trj7;
    Trj[8] = Trj8; Trj[9] = Trj9; Trj[10] = Trj10
    c = {1:'red', 2:'green', 3:'blue', 4:'orange', 5:'yellow', 6:'magenta',7:'black', 8:'cyan', 9:'sandybrown', 10:'purple'}
    print('Default setting loaded.') 
    Trj_fn = {}
    for i in range(1,11):
      Trj_fn[i] = "---"


 # Example:    https://docs.bokeh.org/en/latest/docs/gallery/stocks.html
def ppk_plot(t, x, y, z, title, x_title, y_title):
  radii = .1  
  p1 = figure( title=title )                   #
  p1.xaxis.axis_label = x_title
  p1.yaxis.axis_label = y_title
  p1.circle_cross(x,y, size=1)    # Plot the lat, lon
  
  p2 = figure()                   # 
  p2.circle_cross(                          # Plot the elevations vs time
      pd.to_datetime(ppk_data['hms_z']),
      ppk_data['elev'], 
      size=1
      )

  show( 
      gridplot([[p1,p2]], 
              plot_width=settings['plot_width'], 
              plot_height=settings['plot_height']
              ) 
      );

def no_ppk_data_loaded():
  print('No ppk_data is loaded.')


def open_new_plot():
  try:
    webgl
  except:
    NameError
    WebGL = "Enabled"

  if WebGL == 'Enabled':
    p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
               output_backend = "webgl",
                     tools=TOOLS
              )
  else:
      p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
                     tools=TOOLS
              )
  p0.toolbar.autohide                 = True
  p0.title.text_font_size             = '18pt'
  p0.yaxis.axis_label_text_font_size  = '16pt'
  p0.yaxis.major_label_text_font_size = '14pt'
  p0.xaxis.major_label_text_font_size = '14pt'
  p0.xaxis.axis_label_text_font_size  = '16pt'
  return p0

def open_time_plot():
  p0 = open_new_plot()
  p0.yaxis.axis_label                 = "Elevation (meters)"
  p0.xaxis.axis_label                 = "Time"
  p0.xaxis.major_label_orientation    = 1.
  p0.xaxis.formatter = bkm.DatetimeTickFormatter(hours=['%H:%M:%S'], 
                                                minutes=['%H-%M-%S'], 
                                                seconds=['%H_%M_%S']
                                                )
  return p0


def no_ppk_data_loaded():
  print('No ppk_data is loaded.')

# Display:
#  Min Max Nsats
def gen_header( g ):
  headers = { 1:'Number',    2:'File Name', 3:'Records', 4:'Seconds\nOffset', 5:'Start Time',
              6:'Stop Time', 7:'Duration',  8:'Generating\nSoftware', 9:'Trajectory\nType', 10:'User\nZ Bias'}
  for i in range(1,11):
    with g.output_to(0,i):
      print(headers[i])
  return g

def gen_record(g, t, n):
  r = t[n]
  with g.output_to(1,1):
    print(n)
  with g.output_to(1,2):
    print(os.path.split(r['ifn'])[1])
  with g.output_to(1,3):
    print(r['data']['lat'].count())
  with g.output_to(1,4):
    print(r['Seconds_Offset'])
  with g.output_to(1,5):
    print(r['data']['hms_z'].min())
  with g.output_to(1,6):
    print(r['data']['hms_z'].max())
  with g.output_to(1,7):
    print(r['data']['hms_z'].max() - r['data']['hms_z'].min())  
  with g.output_to(1,8):
    print(r['Generating_Software'])
  with g.output_to(1,9):
    print(r['Trajectory_Type'])
  with g.output_to(1,10):
    print(r['z_bias'])

def display_trj_stats( t, n ):
  grid = gwidgets.Grid(2,11, header_row=True, header_column=True)
  gen_header(grid)
  gen_record(grid, t, n)

def display_all_trj_stats():
  grid = gwidgets.Grid(8,11, header_row=True, header_column=True)
  gen_header(grid)
  for i in range(1,10):
    t = trajectories[i]
    if 'ifn' in t:
      gen_record(grid, trajectories, i )

# Example:    https://docs.bokeh.org/en/latest/docs/gallery/stocks.html
def ppk_plot(t, x, y, z, title='title', x_title='xtitle', y_title='ytitle'):
  radii = .1  
  p1 = figure( title=title )                   #
  p1.xaxis.axis_label = x_title
  p1.yaxis.axis_label = y_title
  p1.circle_cross(x,y, size=1)    # Plot the lat, lon
  
  p2 = figure()                   # 
  p2.circle_cross(                          # Plot the elevations vs time
      pd.to_datetime(ppk_data['hms_z']),
      ppk_data['elev'], 
      size=1
      )

  show( 
      gridplot([[p1,p2]], 
              plot_width=settings['plot_width'], 
              plot_height=settings['plot_height']
              ) 
      );

TOOLS = 'pan,wheel_zoom,hover,box_zoom,reset,undo, redo'
TOOLS = 'pan,wheel_zoom,box_zoom,reset,undo, redo, save'
def open_new_plot(width=1000, height=500, tools=TOOLS):
  try:
    settings
  except:
    NameError
    settings = {}
    settings['plot_width'] = width
    settings['plot_height'] = height

  try:
    webgl
  except:
    NameError
    WebGL = "Enabled"

  if WebGL == 'Enabled':
    p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
               output_backend = "webgl",
                     tools=TOOLS
              )
  else:
      p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
                     tools=TOOLS
              )
  p0.toolbar.autohide                 = True
  p0.title.text_font_size             = '18pt'
  p0.yaxis.axis_label_text_font_size  = '16pt'
  p0.yaxis.major_label_text_font_size = '14pt'
  p0.xaxis.major_label_text_font_size = '14pt'
  p0.xaxis.axis_label_text_font_size  = '16pt'
  return p0

def open_time_plot(width=1000, height=500, tools=TOOLS):
  p0 = open_new_plot(width, height)
  p0.yaxis.axis_label                 = "Elevation (meters)"
  p0.xaxis.axis_label                 = "Time"
  p0.xaxis.major_label_orientation    = 1.
  p0.xaxis.formatter = bkm.DatetimeTickFormatter(hours=['%H:%M:%S'], 
                                                minutes=['%H-%M-%S'], 
                                                seconds=['%H_%M_%S']
                                                )
  return p0

if __name__ == '__main__':
  output_notebook()
  Google_Maps_Key = "Replace this with your Google Maps API Key"
  init_analysis()
  print('The Bokeh Toolbox is ready for use.')
  ########################################################
  #   END configuring the plotting tools                 #
  ########################################################
#--------------------xxx

#display_all_trj_stats()
#display_trj_stats( trajectories, 1)


# Trajectory Tools

## Load GNSS Trajectories


### PPP-CA Trajectory Files.


In [ ]:
#@title **PPP-CA:** Load Trajectory {display-mode: "form"}
#@markdown Use the [free online PPP processing software from the Natural Resources Canada  ](https://webapp.geod.nrcan.gc.ca//geod/account-compte/login.php?locale=en "Free PPP") server to generate a precision GNSS trajectory from your RINEX UAS data.

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  traj_target = 1 #@param [1,2,3,4,5,6,7] {type:"raw"}
  User_Z_Bias = 0.0 #@param {type:"raw"}
  PPP_Seconds_Offset = -16 #@param {type:"integer"}
  ppp_pos_ifn = "/content/2019-0725-GP212112-ppp-ca.pos" #@param {type:"string"} 
  Trajectory_Type = "PPP" #@param ["PPP"] {allow-input: true}


  #============================================================================
  # Canadian PPP *.pos file reader
  #============================================================================
  init_analysis()
  f = pathlib.Path(ppp_pos_ifn)
  if f.exists() == False:
    print(f," does not exist.")
  else:
    ppp_pos = pd.read_csv(ppp_pos_ifn, skiprows=5, sep='\s+' )
    Trj_fn[traj_target] = ppp_pos_ifn
    ppk_data = {}
    ppk_data['hms_z']         = pd.to_datetime( ppp_pos['YEAR-MM-DD'] +" "+ppp_pos['HR:MN:SS.SS'] ) \
                                + pd.to_timedelta(PPP_Seconds_Offset, unit='seconds')
    ppk_data['lat']         =     ppp_pos['LATDD']  + ppp_pos['LATMN']/60.0 + ppp_pos['LATSS']/3600.0
    ppk_data['lon']         = -( abs(ppp_pos['LONDD']) + ppp_pos['LONMN']/60.0 + ppp_pos['LONSS']/3600.0 )
    ppk_data['elev']        = ppp_pos['HGT(m)'] + User_Z_Bias
    ppk_data['sdu95']       = ppp_pos['SDHGT(95%)']
    ppk_data['gdop']        = ppp_pos['GDOP']
    ppk_data['ns']          = ppp_pos['NSV']
    ppk_data['q']           = 6;
  

    trajectories[traj_target]['Generating_Software'] = "Can PPP"
    trajectories[traj_target]['Seconds_Offset']      = PPP_Seconds_Offset
    trajectories[traj_target]['Trajectory_Type']     = Trajectory_Type
    trajectories[traj_target]['ifn']  = ppp_pos_ifn
    trajectories[traj_target]['z_bias']  = User_Z_Bias
    trajectories[traj_target]['data'] = ppk_data

    z = ppk_data['elev']
    display_all_trj_stats()


    


### GrafNav Trajectory Files.

In [ ]:
#@title **GrafNav:**  Load File.

def load_grafnav_trj( fn ):
  data = pd.DataFrame()
  ppk_trj = pd.read_csv(fn,
                    header=None, 
                    skiprows=40,
                    index_col=False, 
                    infer_datetime_format=True,
                    parse_dates=[['date','utc']],
                    delim_whitespace=True,
                    names=['UTMeasting', 'UTMnorthing', 'navd88', 'GPS_lat', 'GPS_lon', 'GPS_nad83h', 'Q', 'sdu', 'sdne', 'date', 'utc' ],
                    skipinitialspace=True )
  ppk_trj['Ztime'] = ppk_trj['date_utc']
  ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
  ppk_trj = ppk_trj.set_index('Ztime_idx')
  data['hms_z'] = ppk_trj['Ztime']
  data['lat']   = ppk_trj['GPS_lat']
  data['lon']   = ppk_trj['GPS_lon']
  data['elev'] = ppk_trj['GPS_nad83h'] + User_Z_Bias
  data['q']    = ppk_trj['Q']
  data['sdu95'] = ppk_trj['sdu']*1.95
  data['z_bias']    = User_Z_Bias

  trajectories[traj_target]['Generating_Software'] = "GrafNav"
  trajectories[traj_target]['Trajectory_Type'] = Trajectory_Type
  trajectories[traj_target]['Seconds_Offset']    = GrafNav_Seconds_Offset
  trajectories[traj_target]['z_bias']  = User_Z_Bias
  trajectories[traj_target]['ifn']  = fn
  trajectories[traj_target]['data'] = data
#    print( trajectories[traj_target]['ifn'],
#           trajectories[traj_target]['data']['lat'].count()
#          )
  return

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  traj_target = 1 #@param [1,2,3,4,5,6,7] {type:"raw"}
  User_Z_Bias = 0.0 #@param {type:"raw"}
  GrafNav_Seconds_Offset = 0 #@param {type:"integer"}
  GrafNav_File_Name = "" #@param {type:"string"}
  Trajectory_Type = "PPP" #@param ["PPK Single Base", "PPK Multi Base", "PPP", "RTK", "CA Code", "CA Code + WAAS"] {allow-input: true}
 

  f = pathlib.Path(GrafNav_File_Name)
  if f.exists() == False:
    print(f," does not exist.")
  else:
    # Trj_fn[traj_target] = GrafNav_File_Name
    load_grafnav_trj(GrafNav_File_Name)
    #display_trj_stats( trajectories, traj_target )
    display_all_trj_stats()


### RTKLIB Trajectory Files.

In [ ]:
#@title **RTKLib:** Load File.
traj_target = 2 #@param [1,2,3,4,5,6,7] {type:"raw"}
RTKLIB_Seconds_Offset = 0 #@param {type:"integer"}
User_Z_Bias = 0.0 #@param {type:"raw"}

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  """
  Loads a .pos "Position file" generated by RTKLIB into the pandas dataframe ppk_trj.  The .pos file
  is expected to contain UTC time and not GPS time.
  """
  PPK_File_Name = "paste ppk file name here."
  PPK_File_Name = "/content/Gdrive/Missions/2022-0112-Nardi-Md/2022-0112-Nardi-Md_GNSS/aircraft/trajectories/2022-0112-165831-N7251F-GP165812-MDAI-UK15N-cmb-pos.txt" #@param {type:"string"}
  Trajectory_Type = "PPK" #@param ["PPK", "RTK", "PPP", "Code", "Code+SBAS"] {allow-input: true}


  f = pathlib.Path(PPK_File_Name)
  if f.exists() == False:
    print(f," does not exist.")
  else:
    Trj_fn[traj_target]=PPK_File_Name
    ppk_trj = pd.read_csv( PPK_File_Name, 
                        names=['date', 'zhms', 
                                'GPS_lat', 'GPS_lon', 'GPS_nad83h','Q', 'ns',
                                'sdn', 'sde', 'sdu', 'sdne', 'sdeu', 'sdun', 'age', 'ratio'],
                        delim_whitespace=True,
                        comment='%',
                        skiprows = 1 )
    # ppk_trj
    ppk_trj['Ztime'] = pd.to_datetime( ppk_trj['date']+' '+ppk_trj['zhms']) \
                      + + pd.to_timedelta(RTKLIB_Seconds_Offset, unit='seconds')
    ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
    ppk_trj = ppk_trj.set_index('Ztime_idx')

    data = pd.DataFrame()
    data['hms_z'] = ppk_trj['Ztime']
    data['lat']   = ppk_trj['GPS_lat']
    data['lon']   = ppk_trj['GPS_lon']
    data['elev']  = ppk_trj['GPS_nad83h'] + User_Z_Bias
    data['ns']    = ppk_trj['ns']
    data['sdu95'] = ppk_trj['sdu']*1.95
    data['q']     = ppk_trj['Q']
    data['gdop']  = 0
    trajectories[traj_target]['ifn'] = PPK_File_Name
    trajectories[traj_target]['data'] = data
    trajectories[traj_target]['z_bias']  = User_Z_Bias
    trajectories[traj_target]['Generating_Software'] = "RTKlib"
    trajectories[traj_target]['Trajectory_Type'] = Trajectory_Type
    trajectories[traj_target]['Seconds_Offset']    = RTKLIB_Seconds_Offset
    #display_trj_stats( trajectories, traj_target )
    display_all_trj_stats()

   # %load_ext google.colab.data_table
   # display( data )
   # %unload_ext google.colab.data_table




In [ ]:
#@title Display loaded Trajectories
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()

In [ ]:
#@title Remove Selected Trajectory from Memory
Trajector_to_Remove = 5 #@param ["1", "2", "3", "4", "5", "6", "7"] {type:"raw"}
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  print(Trajector_to_Remove)
  trajectories[Trajector_to_Remove] = {}
  print("Trajectory # ", Trajector_to_Remove, " Removed from Memory.")
  display_all_trj_stats()

# Plots and Graphs

## Plot options & Settings

In [ ]:
#@title 'Settings and Options' { run: "auto" }

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  try:
      settings
  except NameError:
      settings = {}
  #@markdown ---
  Google_Maps_Key = "Replace this with your Google Maps API Key" #@param {type:"string"}
  WebGL =  "Enabled" #@param ["Enabled", "Disabled"]
  #@markdown ---
  #@markdown #Trajectories to Plot 
  Trj1 = True #@param {type:"boolean"}
  Trj2 = True #@param {type:"boolean"}
  Trj3 = True #@param {type:"boolean"} 
  Trj4 = True #@param {type:"boolean"}
  Trj5 = True #@param {type:"boolean"}
  Trj6 = True #@param {type:"boolean"}
  Trj7 = True #@param {type:"boolean"}
  #@markdown ---
  #@markdown  **Plot Options**
  plot_dots = False #@param {type:"boolean"}
  plot_lines = True #@param {type:"boolean"}

  #@markdown ---
  #@markdown **Graphic Window Size (Pixels)**
  Width = 900 #@param {type:"slider", min:200, max:1000, step:10}
  Height = 500 #@param {type:"slider", min:200, max:1000, step:10}
  settings['plot_width'] =  Width
  settings['plot_height'] = Height
  Line_Width = 2 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
  #@markdown ---
  #@markdown **Histogram Bin size** 

  binsz = binSize_In_cm / 100.0
  binSize_In_cm = 2.5 #@param {type:"slider", min:1, max:100, step:0.5}
  Trj[1] = Trj1
  Trj[2] = Trj2
  Trj[3] = Trj3
  Trj[4] = Trj4
  Trj[5] = Trj5
  Trj[6] = Trj6
  Trj[7] = Trj7

  print('Operation completed.')
  

## Plots

### Plot Lat vs Lon

In [ ]:
#@title Plot lat vs Lon

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  Lat_Lon_Title = "lat / Lon" #@param {type:"string"}
  display_all_trj_stats()
  p0 = open_new_plot()
  p0.title.text = Elevation_Plot_Title = Lat_Lon_Title
  p0.yaxis.axis_label = "Latitude"
  p0.xaxis.axis_label = "Longitude"
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print("in ",T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
      dt =  trajectories[T]['data']
      if plot_lines:
        p0.line(         
          dt['lon'],
          dt['lat'],
          color=c[T],
          legend=str(T),
          line_width=2
          )
      if plot_dots:
        p0.circle_cross(                          # Plot the elevations vs time
          dt['lon'],
          dt['lat'], 
          size=4,
          legend=str(T),
          color=c[T]
          )
    #else:
            #print("else ", T, c[T],Trj[T], Trj_fn[T])
  show(p0)

### Plot elevation differences between trajectories.

In [ ]:
#@title **Plot** Elevation Differences: Trj1-Trj[n]
#@markdown Y axis limits (m)
dif_range = 0.5 #@param {type:"slider", min:0.25, max:2, step:0.25}

Elevation_Plot_Title = "Elevation Differences: Reference-Trajectory(" #@param {type:"string"}
Reference_Trajectory = 1 #@param [1,2,3,4,5,6,7] {type:"raw"}


import bokeh.plotting as bkp
import bokeh.models as bkm
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  p0 = open_time_plot()
  p0.title.text = Elevation_Plot_Title
  p0.yaxis.axis_label = "Elevation Difference (m)"
  print("The Reference is # ", Reference_Trajectory)

  def plot_difs( p, n, c ):
    mn1=trajectories[Reference_Trajectory]['data']['hms_z'].min() 
    mx1=trajectories[Reference_Trajectory]['data']['hms_z'].max()
    td1=mx1-mn1
    mn2=trajectories[n]['data']['hms_z'].min() 
    mx2=trajectories[n]['data']['hms_z'].max()
    td2=mx2-mn2

    if mn1 < mn2:
      common_start = mn2
    else:
      common_start = mn1

    if mx1 > mx2:
      common_stop = mx2
    else:
      common_stop = mx1

    td1,td2,mn1,mn2,mx1,mx2, common_start, common_stop
    y_interp = np.interp(
        trajectories[Reference_Trajectory]['data']['hms_z'],
        trajectories[n]['data']['hms_z'],
        trajectories[n]['data']['elev']
        )
    y_diff = trajectories[Reference_Trajectory]['data']['elev']-y_interp

    y_diff

    p0.line(                          # Plot the elevations vs time
        pd.to_datetime(trajectories[Reference_Trajectory]['data']['hms_z']),
        y_diff,
        color=c,
        legend=str(Reference_Trajectory)+" - "+str(n),
        )
    p0.y_range=bkm.Range1d(-dif_range-.1,dif_range+.1)
    return

  display_all_trj_stats()
  if Reference_Trajectory in trajectories:
    for T in trajectories:
      if T in trajectories and Trj[T] == True and 'data' in trajectories[T] and 'data' in trajectories[Reference_Trajectory]:
        #print("in ",T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
        #print(T, c[T],  Trj_fn[T])
        if T != Reference_Trajectory:
          plot_difs( p0, T, c[T] )
    show(p0)
  else:
    print("No reference trajectory selected.")



## Plot Number of satelites & height stdev vs time

In [ ]:
#@title Plot Stdev95 vs Time
#@markdown Y axis limits (m)
Elev_range = 0.3 #@param {type:"slider", min:0.1, max:5, step:0.1}
import bokeh.plotting as bkp
import bokeh.models as bkm

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()
  Elevation_Stats = "Elevation Stdev95% & NSATS vs Time " #@param {type:"string"}
  p0 = open_time_plot()
  p0.yaxis.axis_label= "Elevation Std Dev (95%)"
  p0.title.text = Elevation_Stats
  p0.add_layout(bkm.LinearAxis(y_range_name="NSATS"), 'right')
  p0.extra_y_ranges = {"NSATS": bkm.Range1d(start=0, end=20) }
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
      dt =  trajectories[T]['data']
      if plot_dots:
        p0.circle_cross(                         # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['sdu95'], 
        size=4,
        color=c[T]
        )
      if plot_lines:
        p0.line(                                 # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['sdu95'], 
        color=c[T],
        line_width=Line_Width,
        legend="SDU95"
        )
        if 'ns' in dt:
          p0.line(pd.to_datetime(dt['hms_z']),
                dt['ns'],
                color="Black",
                line_width=Line_Width,
                line_dash='dotted',
                legend="NSATS",
                y_range_name="NSATS"
                )
        p0.y_range=bkm.Range1d(0.0,Elev_range+.02)

      p0.y_range=bkm.Range1d(0.0,Elev_range)
  show(p0)

## Plot Elevations


In [ ]:
#@title Plot Elevations vs Time
import bokeh.plotting as bkp
import bokeh.models as bkm

Elevation_Dif_Plot_Title = "Elevations vs Time" #@param {type:"string"}

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()
  p0 = open_time_plot()
  p0.title.text = Elevation_Dif_Plot_Title
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], Trj_fn[T])
      dt =  trajectories[T]['data']
      if plot_dots:
        p0.circle_cross(                          # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['elev'], 
        size=4,
        color=c[T],
        )
      if plot_lines:
        p0.line(                          # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['elev'],
        legend=str(T),
        line_width=Line_Width, 
        color=c[T]
        )

  show(p0)

## Plot Elevation Histograms

In [ ]:
#@title Plot Elevation Histograms
# Based on: https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html

def ppk_zhist(p1, x, z, c, lbl):
  p1.xaxis.axis_label = "Elevation Meters"
  if plot_lines:
    p1.line(x,z, 
            color=c, 
            line_width=Line_Width,
            legend=lbl
            )
  if plot_dots:
    p1.circle(x,z,fill_color='white', color=c, size=8)


def gen_hist( p1, data, c, lbl ):
  data = trajectories[T]['data']
  binsz = settings['binsz']
  z = data['elev']
  a = np.arange(z.min(), z.max(), binsz)
  h, edges = np.histogram( data['elev'], range=(z.min(),z.max()), bins=a.size)
  i = np.where( h == h.max())[0]
  i = i[0]
  edges[i]
  to_z = edges[i]
  ppk_zhist( p1, edges[0:-1], h, c, lbl )

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:

  #print(h.max()," Values found at ", to_z, "m Elevation (m) using ", binsz*100.0, "cm bins.")
  #print("This is the most likely takeoff location.")
  display_all_trj_stats()
  p1 = open_new_plot()
  Elevation_histograms_Title =  "Target. 2.5cm Bins." #@param "ddd" {type:"string"}
  binSize_In_cm = 2.5 #@param {type:"slider", min:1, max:100, step:0.5}
  binsz = binSize_In_cm / 100.0
  settings['binsz'] = binsz
  
  if Elevation_histograms_Title == "":
    p1.title.text =  "Historgrams with "+str(binSize_In_cm)+"cm bins"
  else:
    p1.title.text = Elevation_histograms_Title


  p1.xaxis.axis_label = "Elevation (meters)"
  p1.yaxis.axis_label = "Count"
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], trajectories[T]['ifn'])
      gen_hist( p1, trajectories[T], c[T], str(T) )

  show(p1)


Revision History and TODO List
===

## Revisions
* 2020-1021
  * Integrated changes and code from pix-2-pos version.
* 2019-1109
  * Added cell to delete a selected trajectory from memory.
* 2019-1108
  * Constructed unified statistics display functions
  * fixed various bugs 
  * Added auto-hide for graph menu
  * Added user z_bias so user can offset trajectory elevation if so desired
  * Added additional user input data to help describe each trajectory
*   2019-1106
  * Added stats printout def for read in trajectories
  * Added file names of displayed trjs to printout above each graph.
  * Added User input title to each graph
  * Cleaned up dif plotter
  * Added ability to difference several trj from a reference
  * Preset graph mouse functions for each graphic
  * Improved Trap errors when wrong file loaded, or missing columns
  * Added code to instruct the user to run the initilization cell before using each tool.
*   2019-1005
  * Consolidated the graphic style stuff in a Defs
  * Fixed length difference warnings from some graphs
  * Trapped error when 'ns' column missing from a trajectory in the stats plot
  * Added undo * redo buttons to plots
* 2019-1003
---

## To Do List
1. Translate trajectories to an export trajectory file that the PPK EO tool can read and process.
1. Add histogram for differenced trajectories
1. Add tools to display, select, and delete trajectories 
1. Add numerical statistics to various plots to quantify differences



## Wish List
* Add automatic extraction, and creation of GCPs, of UAS takeoff and landing sites.
* Add reader for CWW-PPK_Geotag output file.
* Add reader for John Sontags trajectory files
* Add reader for NASA GIPSE trajectory files
* Add RTKlib compatible output option so we can drive the CWW-PPK processing software
* Add reader and plotting ability for OPUS result files.
* Add generic xy plotter reader, plotter, difference, etc.  For Global Mapper transect plots.
* Add linux tools to convert raw data to RINEX
* Add ability to GrafNav reader to find the header row, and use it for column labels
* Add background map to the plan view
* Add a switch for UTM on the plan view
* Add stats to histogram plot
* Add reader for Flash shoe time stamps
* Add plotter for Flash shoe timing display with zoom
* Add reader & plotter for EXIF files.
* Integrate [Juypter ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html)

## References
* [Markdown in Colabs](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=70pYkR9LiOV0)
* [ Bokeh interactive visualization library]( https://docs.bokeh.org/en/latest/index.html)
* [Bokeh Interactions, widgets](https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html)
* [Colabs grid widgets](https://colab.research.google.com/notebooks/widgets.ipynb#scrollTo=P6xc9QVFSlrw)
* [os.path for filename manipulation](https://docs.python.org/3/library/os.path.html)
* [Juypter Widget list and examples](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [O'Reilly Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)






---




In [ ]:
#@title Run RTKlib process for Single

!rm -rf /content/junk-P10GN-single.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 0 -m 10 \
-f 2 -h -o junk-P10GN-single.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -50 /content/junk-P10GN-single.txt

In [ ]:
#@title Run RTKlib process for Single

!rm -rf /content/junk-P10GN-single.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 0 -m 10 \
-f 2 -h -o junk-P10GN-single.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -25 /content/junk-P10GN-single.txt

In [ ]:
#@title Run RTKlib process for SBAS

!rm -rf /content/junk-P10GN-sbas.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 1 -m 10 \
-f 2 -h -o junk-P10GN-sbas.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -50 /content/junk-P10GN-sbas.txt

In [ ]:
#@title Run RTKlib process for Single with broadcast

!rm -rf /content/junk-B10GN-single.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux-b.conf -t -u -d 6 -c -p 0 -m 10 \
-f 2 -h -o junk-B10GN-single.txt /content/GP212112.obs  \
 /content/ctmc2060.19n /content/ctmc2060.19g
!head -50 /content/junk-B10GN-single.txt

In [ ]:
#@title Run RTKlib process for Single

!rm -rf /content/junk-P10GN-single.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 0 -m 10 \
-f 2 -h -o junk-P10GN-single.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -50 /content/junk-P10GN-single.txt

In [ ]:
#@title Run RTKlib process for kinematic

!rm -rf /content/junk-P10N-ppk.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 2 -m 10 \
-f 2 -h -o junk-P10GN-ppk.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -25 /content/junk-P10GN-ppk.txt

In [ ]:
#@title Run RTKlib process for PPP
Elevation_Mask = "15" #@param {type:"string"}
print(Elevation_Mask)


!rm -rf /content/junk-P10GN-ppp.txt
!/usr/bin/rnx2rtkp  -k /content/rtklib-linux.conf -t -u -d 6 -c -p 6 -m 10 \
-f 2 -h -o junk-P10GN-ppp.txt /content/GP212112.obs  /content/ctmc2060.19o  \
 /content/ctmc2060.19n /content/ctmc2060.19g /content/igs20634.sp3
!head -25 /content/junk-P10GN-ppp.txt

## RTKlib run help 

# Works in Progress

In [ ]:
#@title Experiment with ip Widgets
import ipywidgets as ipw
from IPython.display import display
button = ipw.Button(description="Click Me!")
cb1 = ipw.Checkbox( value=False, description="Check1", tooltip="A tip here.", button_style='')
cb2 = ipw.Checkbox( value=False, description="Check2", tooltip="A tip here.", button_style='')
cb = {}
for i in range(1,8):
  cb[i] = ipw.Checkbox( value=False, description="Check", tooltip="A tip here.", button_style='')
output = ipw.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Button clicked.")

for i in range(1,8):
  display( cb[i])
button.on_click(on_button_clicked)
display(button, cb1, cb2,  output)

In [ ]:
#@title Generate output suitable for the PPK-geotagging notebook.  Select Trajectory, then run this cell
Selected_Output_Trajectory = 3 #@param ["1", "2","3","4", "5", "6","7"] {type:"raw"}

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  t_out = trajectories[Selected_Output_Trajectory]
  d_out = trajectories[Selected_Output_Trajectory]['data']
  print(t_out['ifn'])
  ofn = str.split( str.split(t_out['ifn'],'/')[-1], '.')[0]+'-TR.csv'
  print('# Orginal Generating Software:', t_out['Generating_Software'] )
  print('#                      Method:', t_out['Trajectory_Type'] )
  print(d_out[{'lat','lon','elev'}])